# streaming bokeh dynamic plots

To be used with the GUI Graph example , I think the GUI window needs to be open ?

The asleep async value needs to be adjusted with the plot refresh rate so that python has bandwidth to receive data + update plot stream

In [1]:
import re
import asyncio
import time
import array
import copy

import numpy as np
import pandas as pd

import serial
import serial.tools.list_ports
from  websockets.sync.client import connect

import bokeh.plotting
import bokeh.io
import bokeh.driving
bokeh.io.output_notebook()

notebook_url = "localhost:8888"

import os
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = notebook_url
#os.environ["BOKEH_ALLOW_WS_ORIGIN"] ="1t4j54lsdj67h02ol8hionopt4k7b7ngd9483l5q5pagr3j2droq"

Loading BokehJS ...

In [2]:
# use with Gui Graph example

ip = "192.168.7.2"
port=5555
address = "gui_data"
notebook_url = "localhost:8888"

# the gui needs to be open for this???

websocket =  connect(f'ws://{ip}:{port}/{address}')

In [3]:
async def daq_stream_async(data,websocket,
                           n_data = 200):
    tmp_data = copy.deepcopy(data)
    channel = None
    n = 0

    while len(data["time_ms"]) < n_data or len(data["voltage"]) < n_data: ## not extrapolating yet
        msg =  websocket.recv()
        n +=1
        if len(msg) == 3:
            channel = int(str(msg)[2])
        else:
            # first channel to come in has to be 0, otherwise taking data from different render iterations
            arr = array.array('f', msg).tolist()        
            if channel == 0:
                tmp_data['time_ms'] += arr
            elif channel == 1:
                tmp_data['voltage'] += arr    
                
            # update time_ms and voltage in one go to avoid feeding data of different lengths to streaming plot
            if len(tmp_data['time_ms']) == len(tmp_data['voltage']):
                data["time_ms"]  = tmp_data["time_ms"][:n_data]
                data["voltage"] = tmp_data["voltage"][:n_data]

            await asyncio.sleep(0.01) # this value? -- adjust to refresh streaming?

    
    return dict({
        "time_ms": data["time_ms"],
        "voltage": data["voltage"]
    })    

## potentiometer app

In [4]:
def potentiometer_app(data, n_data=100, rollover=None, plot_update_delay=90):
    """Return a function defining a Bokeh app for streaming
    data up to `n_data` data points. A maximum of `rollover`
    data points are shown at a time.
    """
    def _app(doc):
        # Instatiate figures
        p = bokeh.plotting.figure(
            frame_width=500,
            frame_height=175,
            x_axis_label="time (s)",
            y_axis_label="voltage (V)",
            y_range=[-0.2, 5.2],
        )

        # No padding on x_range makes data flush with end of plot
        p.x_range.range_padding = 0

        # Start with an empty column data source with time and voltage
        source = bokeh.models.ColumnDataSource({"t": [], "V": []})

        # Put a line glyph
        r = p.line(source=source, x="t", y="V")

        @bokeh.driving.linear()
        def update(step):
            # Shut off periodic callback if we have plotted all of the data
            if step > n_data:
                doc.remove_periodic_callback(pc)
            else:
                # Update plot by streaming in data
                source.stream(
                    {
                        "t": np.array(data['time_ms'][data['prev_array_length']:]) / 1000,
                        "V": data['voltage'][data['prev_array_length']:],
                    },
                    rollover,
                )
                data['prev_array_length'] = len(data['time_ms'])

        doc.add_root(p)
        pc = doc.add_periodic_callback(update, plot_update_delay)
        
        
    return _app

In [5]:
data = dict(
        prev_array_length = 0,
        time_ms = [],
        voltage = []
    )

n_data = 10000

bokeh.io.show(potentiometer_app(data,n_data=n_data, plot_update_delay=50), notebook_url=notebook_url)
daq_task = asyncio.create_task(daq_stream_async(data,websocket,
                           n_data = n_data))

In [50]:
print(len(data["time_ms"]), len(data["voltage"]))

500 500


## test

In [4]:
data_test = dict(
        prev_array_length = 0,
        time_ms = [],
        voltage = []
    )
print("len test" , len(data_test["time_ms"]),len( data_test["voltage"]))

daq_task = asyncio.create_task(daq_stream_async(data_test,n_data=250, websocket=websocket))

len test 0 0


In [5]:
daq_task.done()

False

In [60]:
data_result = daq_task.result()
len(data_result['time_ms']), len(data_result['voltage']), len(data_test['time_ms']),  len(data_test['voltage'])

(250, 250, 250, 250)